In [10]:
# import dependencies
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

In [11]:
#  Import and read the csv.
data_df = pd.read_csv("Triangle_role/Resources/sample_data.csv")
data_df.head()

,Year,FIPS,C_S,month,cases,Pcincome,POPESTIMATE,MFratio,POPEST_MALE,POPEST_FEM,...,WA,BA,IA,AA,NA,TOM,NH,H,Cost,RUCC
0,2017,1001,"Autauga County,Alabama",1,0,27824,55448,0.951913,27041,28407,...,42606,10816,254,693,59,1020,53908,1540,186712,2
1,2017,1001,"Autauga County,Alabama",2,0,27824,55448,0.951913,27041,28407,...,42606,10816,254,693,59,1020,53908,1540,186577,2
2,2017,1001,"Autauga County,Alabama",3,0,27824,55448,0.951913,27041,28407,...,42606,10816,254,693,59,1020,53908,1540,186397,2
3,2017,1001,"Autauga County,Alabama",4,0,27824,55448,0.951913,27041,28407,...,42606,10816,254,693,59,1020,53908,1540,186839,2
4,2017,1001,"Autauga County,Alabama",5,0,27824,55448,0.951913,27041,28407,...,42606,10816,254,693,59,1020,53908,1540,187155,2


In [18]:
object_columns = list(data_df.select_dtypes(['object']).columns)
object_columns

['C_S']

In [19]:
# convert the string values into numerical ones using the get_dummies() method
data_df = pd.get_dummies(data_df, columns=object_columns)
data_df.head()

,Year,FIPS,month,cases,Pcincome,POPESTIMATE,MFratio,POPEST_MALE,POPEST_FEM,MEDIAN_AGE_TOT,...,NA,TOM,NH,H,Cost,RUCC,"C_S_Adams County,Ohio","C_S_Atlantic County,New Jersey","C_S_Autauga County,Alabama","C_S_Beaver County,Pennsylvania"
0,2017,1001,1,0,27824,55448,0.951913,27041,28407,38.8,...,59,1020,53908,1540,186712,2,0,0,1,0
1,2017,1001,2,0,27824,55448,0.951913,27041,28407,38.8,...,59,1020,53908,1540,186577,2,0,0,1,0
2,2017,1001,3,0,27824,55448,0.951913,27041,28407,38.8,...,59,1020,53908,1540,186397,2,0,0,1,0
3,2017,1001,4,0,27824,55448,0.951913,27041,28407,38.8,...,59,1020,53908,1540,186839,2,0,0,1,0
4,2017,1001,5,0,27824,55448,0.951913,27041,28407,38.8,...,59,1020,53908,1540,187155,2,0,0,1,0


In [20]:
# Create our features
X = data_df.drop(columns="Cost")

# Create our target
y = data_df["Cost"]

In [21]:
X.describe()

,Year,FIPS,month,cases,Pcincome,POPESTIMATE,MFratio,POPEST_MALE,POPEST_FEM,MEDIAN_AGE_TOT,...,AA,NA,TOM,NH,H,RUCC,"C_S_Adams County,Ohio","C_S_Atlantic County,New Jersey","C_S_Autauga County,Alabama","C_S_Beaver County,Pennsylvania"
count,240.000000,240.000000,240.000000,2.400000e+02,240.000000,240.000000,240.000000,240.000000,240.000000,240.000000,...,240.000000,240.000000,240.000000,240.000000,240.000000,240.0000,240.000000,240.000000,240.000000,240.000000
mean,2019.000000,29002.500000,6.500000,9.963173e+04,29123.150000,251490.600000,0.967329,124875.550000,126615.050000,40.145000,...,5844.900000,97.100000,3199.100000,115423.300000,14265.700000,2.7500,0.250000,0.250000,0.250000,0.250000
std,1.417169,16451.953836,3.459267,2.441305e+05,4414.061896,169839.609081,0.034158,86578.766423,83311.526627,4.013426,...,9147.700576,94.232633,2803.147578,77361.080483,21904.397695,1.9243,0.433918,0.433918,0.433918,0.433918
min,2017.000000,1001.000000,1.000000,0.000000e+00,20248.000000,55448.000000,0.937129,27041.000000,28407.000000,33.800000,...,83.000000,3.000000,376.000000,27129.000000,283.000000,1.0000,0.000000,0.000000,0.000000,0.000000
25%,2018.000000,25751.000000,3.750000,0.000000e+00,27002.750000,137740.750000,0.944174,66948.000000,70793.000000,37.800000,...,526.500000,45.000000,870.750000,47297.000000,1233.750000,1.7500,0.000000,0.000000,0.000000,0.000000
50%,2019.000000,36501.000000,6.500000,0.000000e+00,29880.000000,215692.500000,0.951635,104868.000000,110824.500000,40.550000,...,803.000000,64.500000,2356.000000,109073.500000,2301.000000,2.0000,0.000000,0.000000,0.000000,0.000000
75%,2020.000000,39752.500000,9.250000,5.480975e+04,31780.750000,332153.750000,0.977301,164243.500000,167910.250000,42.925000,...,6082.000000,112.750000,4904.750000,176516.250000,15032.250000,3.0000,0.250000,0.250000,0.250000,0.250000
max,2021.000000,42007.000000,12.000000,1.363149e+06,36143.000000,522140.000000,1.031982,265179.000000,256961.000000,45.300000,...,22076.000000,299.000000,7912.000000,220232.000000,54734.000000,6.0000,1.000000,1.000000,1.000000,1.000000


In [22]:
# Check the balance of our target values
y.value_counts()

216072    2
150610    2
216613    2
113673    1
108384    1
108226    1
107882    1
107688    1
107834    1
108089    1
108820    1
109595    1
110508    1
110889    1
111392    1
111952    1
186712    1
107772    1
115123    1
117599    1
119515    1
121689    1
123081    1
124383    1
125674    1
126950    1
128459    1
129996    1
131214    1
132082    1
108230    1
107147    1
129659    1
100451    1
95114     1
95467     1
95891     1
96284     1
96558     1
96784     1
97087     1
97611     1
98136     1
98656     1
99223     1
99800     1
101079    1
106457    1
101668    1
102039    1
102339    1
102620    1
102959    1
103340    1
103723    1
104117    1
104362    1
104655    1
105037    1
105664    1
134398    1
130958    1
130398    1
146743    1
147699    1
148745    1
149586    1
145538    1
143288    1
143538    1
144256    1
145292    1
146911    1
148045    1
149519    1
150914    1
152008    1
153318    1
154719    1
156855    1
158602    1
160152    1
161921    1
1637

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=22)
Counter(y_train)

Counter({213784: 1,
         203002: 1,
         275912: 1,
         113673: 1,
         189847: 1,
         145538: 1,
         188323: 1,
         107147: 1,
         143538: 1,
         110889: 1,
         99800: 1,
         208412: 1,
         245806: 1,
         202835: 1,
         191941: 1,
         146569: 1,
         187850: 1,
         187276: 1,
         105037: 1,
         204387: 1,
         111392: 1,
         136139: 1,
         169895: 1,
         106457: 1,
         200999: 1,
         212035: 1,
         209646: 1,
         173980: 1,
         188053: 1,
         150914: 1,
         94439: 1,
         129659: 1,
         189809: 1,
         144926: 1,
         200156: 1,
         171950: 1,
         179789: 1,
         148745: 1,
         165821: 1,
         202351: 1,
         236889: 1,
         186577: 1,
         209394: 1,
         203085: 1,
         198429: 1,
         96558: 1,
         133634: 1,
         136954: 1,
         182351: 1,
         189278: 1,
   

In [24]:
# create a logistical regression model
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs', random_state=22)
classifier

LogisticRegression(random_state=22)

In [25]:
# Train the data
classifier.fit(X_train, y_train)

C:\Users\dog\anaconda3\envs\PythonData\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression(random_state=22)

In [26]:
# Predict outcomes for test data set
predictions = classifier.predict(X_test)
pd.DataFrame({"Prediction": predictions, "Actual": y_test})

,Prediction,Actual
85,194970,197032
46,216072,216072
124,93686,95114
197,142237,139847
15,200156,193615
187,136139,132674
99,194970,206538
32,206885,204749
49,213784,216613
204,150610,143665


In [27]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, predictions)

0.016666666666666666

In [ ]:
#Using drop() to delete rows based on column value
data_df.drop(data_df[data_df['cases'] < 1].index, inplace = True)

# Default value of display.max_rows is 10 so at max
# 10 rows will be printed. Set it None to display
# all rows in the dataframe
pd.set_option('display.max_rows', None)

data_df

In [ ]:
c_s_cost_df = data_df[["C_S","Cost"]]
c_s_cost_df

In [ ]:
for i in c_s_cost_df["C_S"].str.len():
      
    if c_s_cost_df['C_S'].shift() == c_s_cost_df['C_S']:
#    if c_s_cost_df["C_S"][i+1] == c_s_cost_df["C_S"][i]:

#        print(c_s_cost_df["C_S"][i])

        print("I work.")

In [ ]:
sorted_df = data_df.sort_values(['Cost']).groupby(["Year","C_S","month"])["Cost"].sum()
print(sorted_df.to_string())

In [ ]:
# Declare a list that is to be converted into a column
increase_list = []

for year in sorted_df['Year']:

    for countystate in sorted_df["C_S"].str.len():

        for cost in sorted_df["Cost"]:
                
                if cost+1 < cost:
                    increase_list.append("1")
                else:
                    increase_list.append("0")
        data_df["Increase"] = increase_list 

print(data_df)

In [ ]:
# split data into training and testing
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)